XGBoost GBM'in hiz ve tahmin performansini arttirmak uzere optimizie edilmis olcekelnebilir ve farkli platformlara entegre edilebilir halidir

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR


from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
df = pd.read_csv('Hitters.csv')
df = df.dropna()
dms= pd.get_dummies(df[['League','Division','NewLeague']]) #kategorik degiskenleri dummy degiskenlere donusturduk
y=df["Salary"]
X_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')# bu 4 sutun kaldirir kalan sutunlari da float64 veri turune cevirir
X=pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [7]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/5a/0b/f9f815f240a9610d42367172b9f7ef7e8c9113a09b1bb35d4d85f96b910a/xgboost-3.0.0-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.8 MB/s eta 0:00:00a 0:00:01


In [9]:
import xgboost

In [11]:
from xgboost import XGBRegressor

In [15]:
xgb = XGBRegressor().fit(X_train,y_train)

In [17]:
xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [19]:
y_pred = xgb.predict(X_test)

In [21]:
np.sqrt(mean_squared_error(y_test,y_pred))

366.3863437634965

### Model Tuning

In [24]:
xgb_params = {"learning_rate" :[0.1,0.01,0.05],
              "max_depth" : [2,3,4,5,8],
              "n_estimators": [100,200,500,100],
              "colsample_bytree" : [0.4,0.7,1] }

In [26]:
xgb_cv_model = GridSearchCV(xgb,xgb_params , cv = 10 , n_jobs = -1,verbose = 2).fit(X_train,y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


In [28]:
xgb_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.01,
 'max_depth': 8,
 'n_estimators': 500}

In [32]:
xgb_tuned = XGBRegressor(colsample_bytree =  0.4,
 learning_rate =  0.01,
 max_depth = 8,
 n_estimators =  500).fit(X_train,y_train)

In [34]:
y_pred = xgb_tuned.predict(X_test)

In [36]:
np.sqrt(mean_squared_error(y_test,y_pred))

343.39367648154325